# Preprocessing raw mri data
This markdown should guide you through the process of taking raw fMRI data through our preprocessing pipeline. 

All of this code can be edited (or run) in the terminal by accessing the script B3/scripts/preprocess.sh

### Prerequisite steps:
By the time you get here, you should have: 
* Copied the relevant directories over to the B3/mri directory
* Created a directory with your Peds ID within B3/staging_areas

## Step 1: Copying files 

The first thing we'll need to do is copy over the relevant files and rename them. We do this for two reasons: 
1. The raw data file contains many files we don't need for most analyses
2. The raw data is not the same for every participant because of false starts or other errors during the scan 

Before running this command, take a look at the DICOM directories in your raw data files. This script is set up for data with:
* Two self-other directories (00006 and 00007)
* Two peer feedback directories (00008 and 00009)
* Two conformity directories (00010 and 00011)
* One Cyberball directory (00012)
* One anatomical directory (00002)

If your data structure is different from this, consult the scan logs on Box to see which directories contain the scans we'll want to use for analysis (for example --- if you have three self-other directories, you many find that the first directory contains info from a false start). Edit the code accordingly (note that some of the early participants have a different data structure because they only had one run for the first three tasks). 

Once the code is accurate, add all record IDs after "for subj in". Just put a space between each. 

In [1]:
%%bash
for subj in 732; do
mkdir -p ../BIDS_data/sub-${subj}/ses-01/anat ../BIDS_data/sub-${subj}/ses-01/func
cp ../mri/${subj}/dicoms/00006.task_Self_Other/00006.task_Self_Other.tgz ../BIDS_data/sub-${subj}/ses-01/func/1_Self_Other.tgz
cp ../mri/${subj}/dicoms/00007.task_Self_Other/00007.task_Self_Other.tgz ../BIDS_data/sub-${subj}/ses-01/func/2_Self_Other.tgz
cp ../mri/${subj}/dicoms/00008.task_Peer_Feedback/00008.task_Peer_Feedback.tgz ../BIDS_data/sub-${subj}/ses-01/func/1_Peer_Feedback.tgz
cp ../mri/${subj}/dicoms/00009.task_Peer_Feedback/00009.task_Peer_Feedback.tgz ../BIDS_data/sub-${subj}/ses-01/func/2_Peer_Feedback.tgz
cp ../mri/${subj}/dicoms/00010.task_Conformity/00010.task_Conformity.tgz ../BIDS_data/sub-${subj}/ses-01/func/1_Conformity.tgz
cp ../mri/${subj}/dicoms/00011.task_Conformity/00011.task_Conformity.tgz ../BIDS_data/sub-${subj}/ses-01/func/2_Conformity.tgz
cp ../mri/${subj}/dicoms/00012.task_Cyberball/00012.task_Cyberball.tgz ../BIDS_data/sub-${subj}/ses-01/func/1_Cyberball.tgz
cp ../mri/${subj}/dicoms/00002.3D_Sag_MPRAGE/00002.3D_Sag_MPRAGE.tgz ../BIDS_data/sub-${subj}/ses-01/anat/1_Anat.tgz
done

## Step 2: Converting files to BIDS format

This section of the code actually does the work of converting all of the files into the BIDS format needed for fmriprep. You can learn more about the BIDS format [here](https://bids.neuroimaging.io/).

This should take a while. If your participant only has one run for any of the given tasks, you'll need to edit this code to reflect that. 

For each file, this basically follows these steps:
1. Creates a temporary directory
2. Unzips the .tgz file that contains all slices to that temporary directory and deletes it 
3. Combines all slices into a single NIFTI (.nii) file with a JSON (.json) sidecar (that file contains important info about the scan)
4. Deletes the temporary directory
5. Renames the NIFTI and JSON files to reflect the BIDS format 

As before, you'll need to edit the actual subjects. 

You might get a weird message in this notebook after running this that looks like an error. It probably worked -- just check the directory to see. 

In [4]:
%%bash

for subj in 732; do 

cd ../BIDS_data/sub-${subj}/ses-01/func

tempdir=$(mktemp -d)
tar -zxvf 1_Self_Other.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-selfother_run-01_bold.nii
mv tmp*json sub-${subj}_ses-01_task-selfother_run-01_bold.json

tempdir=$(mktemp -d)
tar -zxvf 2_Self_Other.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-selfother_run-02_bold.nii
mv tmp*json sub-${subj}_ses-01_task-selfother_run-02_bold.json

tempdir=$(mktemp -d)
tar -zxvf 1_Peer_Feedback.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-peerfeedback_run-01_bold.nii
mv tmp*json sub-${subj}_ses-01_task-peerfeedback_run-01_bold.json

tempdir=$(mktemp -d)
tar -zxvf 2_Peer_Feedback.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-peerfeedback_run-02_bold.nii
mv tmp*json sub-${subj}_ses-01_task-peerfeedback_run-02_bold.json

tempdir=$(mktemp -d)
tar -zxvf 1_Conformity.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-conformity_run-01_bold.nii
mv tmp*json sub-${subj}_ses-01_task-conformity_run-01_bold.json

tempdir=$(mktemp -d)
tar -zxvf 2_Conformity.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-conformity_run-02_bold.nii
mv tmp*json sub-${subj}_ses-01_task-conformity_run-02_bold.json

tempdir=$(mktemp -d)
tar -zxvf 1_CyberBall.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_task-cyberball_run-01_bold.nii
mv tmp*json sub-${subj}_ses-01_task-cyberball_run-01_bold.json

rm -f *tgz

cd ../anat

tempdir=$(mktemp -d)
tar -zxvf 1_Anat.tgz -C "$tempdir"
dcm2niix_afni -o . "$tempdir"
rm -r "$tempdir"
mv tmp*nii sub-${subj}_ses-01_T1w.nii
mv tmp*json sub-${subj}_ses-01_T1w.json

rm -f *tgz
cd ../../../../scripts

done

Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)
Chris Rorden's dcm2niiX version v1.0.20211006  GCC5.4.0 x86-64 (64-bit Linux)


tar (child): 1_Self_Other.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
Error: Unable to find any DICOM images in /tmp/tmp.62vCYONQAn (or subfolders 5 deep)
mv: cannot stat 'tmp*nii': No such file or directory
mv: cannot stat 'tmp*json': No such file or directory
tar (child): 2_Self_Other.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
Error: Unable to find any DICOM images in /tmp/tmp.7C9s9aKCyS (or subfolders 5 deep)
mv: cannot stat 'tmp*nii': No such file or directory
mv: cannot stat 'tmp*json': No such file or directory
tar (child): 1_Peer_Feedback.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
Error: Unable to find any DICOM im

## Step 3: Copying the data to your staging area

Next we'll just copy this data to a staging area, so we can run fmriprep on just 1-3 participants at a time. In my experience, we run into problems whenever we try to run larger batches (a more savvy user might be able to find a way around this).

To use the code below, just pop in the right record IDs for your subjects and replace ${YOUR_PEDS_ID} with... well... your peds ID. 

In [8]:
%%bash
for subj in 732; do
cp -r ../BIDS_data/sub-${subj} ../staging_areas/${YOUR_PEDS_ID}
done

## Step 4: Running fmriprep (DO THIS IN TERMINAL)

Next, we run fmriprep from the Poldrack labs docker container! This code will run fmriprep for all participants in your staging area, then will place a QA report and the preprocessed data in your home directory. It will take a LONG time -- run it and then check back tomorrow. 

Because it depends on Docker, you probably won't be able to run this command in this notebook. I recommend running this command verbatim directly from the terminal in the B3/scripts directory --- you'll just need to drop in your peds ID. If you decide to run step_one_preprocess.sh in the future, note that it contains this command.  

#### Notes:
We're pretty much using the fmriprep tool right out of the box here --- that's something that might be changed at a future date. You can learn more about the tool at [fmriprep.org](https://fmriprep.org/).

Another important note is that we are currently *not* running a slice time correction as part of our preprocessing (double check this in the QA report). That's probably advisable given the high likelihood for participant motion, but it's something we'll need to address and explain in any methods section. 

In [15]:
%%bash
fmriprep-docker --fs-license-file ../../Software/license.txt ../staging_areas/(YOUR_PEDS_ID) /home/(YOUR_PEDS_ID)@pediatrics.wisc.edu

SyntaxError: invalid syntax (3550566003.py, line 4)

## That's it! 

Congrats! You've just preprocessed some raw fMRI data. As a next step, check out the post-fmriprep notebook in this directory. 

In the future, you can run all of these scripts with the command step_one_preprocess.sh, which is in this directory. Just edit the code slightly to add your subj IDs and peds ID (open with nano step_one_preprocess.sh), then run it with ./step_one_preprocess.sh. Before doing this, check the bottom of the script to make sure the actual fmriprep code is or isn't commented out (preceded by #), depending on whether you want to run this step. 

Thanks and cheers,

Matt